In [ ]:
!pip install langchain langchain-openai rank_bm25 faiss-cpu sentence-transformers langchain-community torch

In [2]:
from langchain_openai import ChatOpenAI
from langchain.schema import Document, HumanMessage
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import CrossEncoder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import torch
import numpy as np

In [3]:
docs_data = [

    {
        "chunk_id": "voc_surveys_1",
        "feature_name": "Multichannel Survey Deployment",
        "category": "Voice of Customer",
        "use_case": "Khảo sát đa kênh",
        "embedding_text": "Thiết kế và gửi khảo sát qua Web, Mobile, Zalo, SMS, Email, QR, và POS để thu thập phản hồi từ khách hàng một cách linh hoạt.",

    },
    {
        "chunk_id": "voc_reviews_1",
        "feature_name": "Review Aggregation & Analysis",
        "category": "Voice of Customer",
        "use_case": "Tổng hợp và phân tích đánh giá",
        "embedding_text": "Thu thập đánh giá khách hàng từ nhiều nền tảng như Google, Facebook và phân tích xu hướng, từ khóa và mức độ hài lòng.",

    },
    {
        "chunk_id": "voc_conversations_1",
        "feature_name": "Conversational Analytics",
        "category": "Voice of Customer",
        "use_case": "Phân tích hội thoại",
        "embedding_text": "Lắng nghe và phân tích nội dung chat, cuộc gọi, và email để rút ra chủ đề chính, xu hướng và mức độ hài lòng.",

    },
    {
        "chunk_id": "ai_inbox_1",
        "feature_name": "AI Inbox",
        "category": "AI Customer Service",
        "use_case": "Hợp tác giữa AI và con người trong hỗ trợ",
        "embedding_text": "Kết hợp AI và con người trong trung tâm liên lạc, giúp xử lý nhanh các yêu cầu, phản hồi tự động và gợi ý hành động.",

    },
    {
        "chunk_id": "ai_tickets_1",
        "feature_name": "Ticket Management System",
        "category": "AI Customer Service",
        "use_case": "Quản lý yêu cầu hỗ trợ",
        "embedding_text": "Hệ thống quản lý yêu cầu hỗ trợ khách hàng với phân loại, ưu tiên và xử lý nhanh chóng nhờ AI.",

    },
    {
        "chunk_id": "insights_experience_1",
        "feature_name": "Experience Insights",
        "category": "Insights",
        "use_case": "Phân tích trải nghiệm khách hàng",
        "embedding_text": "Hiểu rõ hành vi và trải nghiệm khách hàng tại từng điểm chạm bằng phân tích chủ đề và xu hướng phản hồi.",

    },
    {
        "chunk_id": "insights_operation_1",
        "feature_name": "Operational Monitoring",
        "category": "Insights",
        "use_case": "Giám sát hoạt động dịch vụ",
        "embedding_text": "Theo dõi hiệu suất khảo sát, chiến dịch, trung tâm liên hệ và xử lý ticket để đảm bảo hiệu quả vận hành.",

    },
    {
        "chunk_id": "customer_360_customers_1",
        "feature_name": "Customer Database & Segmentation",
        "category": "Customer 360",
        "use_case": "Quản lý và phân loại khách hàng",
        "embedding_text": "Tập trung toàn bộ dữ liệu khách hàng và phân nhóm theo hành vi, giá trị, lịch sử tương tác.",

    },
    {
        "chunk_id": "customer_360_engagement_1",
        "feature_name": "Customer Engagement Campaigns",
        "category": "Customer 360",
        "use_case": "Chạy chiến dịch tương tác khách hàng",
        "embedding_text": "Tạo và triển khai chiến dịch chăm sóc khách hàng đa kênh để tăng tương tác và giữ chân khách hàng.",

    },
    {
        "chunk_id": "ai_automation_1",
        "feature_name": "AI Model & Automation Config",
        "category": "AI & Automation",
        "use_case": "Cấu hình AI và workflow tự động",
        "embedding_text": "Thiết lập mô hình AI, công cụ automation và các điều kiện trigger để tự động hóa quy trình tương tác với khách.",

    }
]

# Chuyển dữ liệu thành đối tượng Document của LangChain
docs = [
    Document(
        page_content=doc["embedding_text"],
        metadata={
            "chunk_id": doc["chunk_id"],
            "feature_name": doc["feature_name"],
            "category": doc["category"],
            "use_case": doc["use_case"],
        }
    ) for doc in docs_data
]

In [4]:
docs

[Document(metadata={'chunk_id': 'voc_surveys_1', 'feature_name': 'Multichannel Survey Deployment', 'category': 'Voice of Customer', 'use_case': 'Khảo sát đa kênh'}, page_content='Thiết kế và gửi khảo sát qua Web, Mobile, Zalo, SMS, Email, QR, và POS để thu thập phản hồi từ khách hàng một cách linh hoạt.'),
 Document(metadata={'chunk_id': 'voc_reviews_1', 'feature_name': 'Review Aggregation & Analysis', 'category': 'Voice of Customer', 'use_case': 'Tổng hợp và phân tích đánh giá'}, page_content='Thu thập đánh giá khách hàng từ nhiều nền tảng như Google, Facebook và phân tích xu hướng, từ khóa và mức độ hài lòng.'),
 Document(metadata={'chunk_id': 'voc_conversations_1', 'feature_name': 'Conversational Analytics', 'category': 'Voice of Customer', 'use_case': 'Phân tích hội thoại'}, page_content='Lắng nghe và phân tích nội dung chat, cuộc gọi, và email để rút ra chủ đề chính, xu hướng và mức độ hài lòng.'),
 Document(metadata={'chunk_id': 'ai_inbox_1', 'feature_name': 'AI Inbox', 'category

In [ ]:
pain_point = "Khách hàng phàn nàn về việc khó hiểu hành vi và trải nghiệm của họ tại các điểm chạm."
query = pain_point  # Sử dụng trực tiếp pain point làm query

In [6]:
#Khởi tạo BM25 Retriever
print("=== Bước 1: Truy xuất tài liệu bằng BM25 ===")
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 6  # Lấy top 5 tài liệu
bm25_docs = bm25_retriever.get_relevant_documents(query)
for i, doc in enumerate(bm25_docs, 1):
    print(f"BM25 Doc {i}: {doc.page_content} (Feature: {doc.metadata['feature_name']})")



=== Bước 1: Truy xuất tài liệu bằng BM25 ===
BM25 Doc 1: Hiểu rõ hành vi và trải nghiệm khách hàng tại từng điểm chạm bằng phân tích chủ đề và xu hướng phản hồi. (Feature: Experience Insights)
BM25 Doc 2: Lắng nghe và phân tích nội dung chat, cuộc gọi, và email để rút ra chủ đề chính, xu hướng và mức độ hài lòng. (Feature: Conversational Analytics)
BM25 Doc 3: Kết hợp AI và con người trong trung tâm liên lạc, giúp xử lý nhanh các yêu cầu, phản hồi tự động và gợi ý hành động. (Feature: AI Inbox)
BM25 Doc 4: Tạo và triển khai chiến dịch chăm sóc khách hàng đa kênh để tăng tương tác và giữ chân khách hàng. (Feature: Customer Engagement Campaigns)
BM25 Doc 5: Tập trung toàn bộ dữ liệu khách hàng và phân nhóm theo hành vi, giá trị, lịch sử tương tác. (Feature: Customer Database & Segmentation)
BM25 Doc 6: Hệ thống quản lý yêu cầu hỗ trợ khách hàng với phân loại, ưu tiên và xử lý nhanh chóng nhờ AI. (Feature: Ticket Management System)


C:\Users\HP\AppData\Local\Temp\ipykernel_19996\1693885745.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  bm25_docs = bm25_retriever.get_relevant_documents(query)


In [7]:
#Khởi tạo Dense Retriever với Sentence Transformers
print("\n=== Bước 2: Truy xuất tài liệu bằng Dense Retrieval ===")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda"} if torch.cuda.is_available() else {"device": "cpu"}
)
vectorstore = FAISS.from_documents(docs, embeddings)
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 6})
dense_docs = dense_retriever.get_relevant_documents(query)
docs_and_scores_for_print = vectorstore.similarity_search_with_score(query, k=6)

for i, (doc, score) in enumerate(docs_and_scores_for_print, 1):
    print(f"Dense Doc {i}: Score={score:.4f} - {doc.page_content} (Feature: {doc.metadata['feature_name']})")
print("--------------------------------------------------")


=== Bước 2: Truy xuất tài liệu bằng Dense Retrieval ===


C:\Users\HP\AppData\Local\Temp\ipykernel_19996\2558086555.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Dense Doc 1: Score=0.5762 - Thu thập đánh giá khách hàng từ nhiều nền tảng như Google, Facebook và phân tích xu hướng, từ khóa và mức độ hài lòng. (Feature: Review Aggregation & Analysis)
Dense Doc 2: Score=0.6146 - Tạo và triển khai chiến dịch chăm sóc khách hàng đa kênh để tăng tương tác và giữ chân khách hàng. (Feature: Customer Engagement Campaigns)
Dense Doc 3: Score=0.6411 - Hiểu rõ hành vi và trải nghiệm khách hàng tại từng điểm chạm bằng phân tích chủ đề và xu hướng phản hồi. (Feature: Experience Insights)
Dense Doc 4: Score=0.6934 - Hệ thống quản lý yêu cầu hỗ trợ khách hàng với phân loại, ưu tiên và xử lý nhanh chóng nhờ AI. (Feature: Ticket Management System)
Dense Doc 5: Score=0.7266 - Tập trung toàn bộ dữ liệu khách hàng và phân nhóm theo hành vi, giá trị, lịch sử tương tác. (Feature: Customer Database & Segmentation)
Dense Doc 6: Score=0.7695 - Thiết kế và gửi khảo sát qua Web, Mobile, Zalo, SMS, Email, QR, và POS để thu thập phản hồi từ khách hàng một cách linh hoạt. (Fe

In [8]:
#Kết hợp BM25 và Dense Retrieval
print("\n=== Bước 3: Kết hợp tài liệu (Ensemble) ===")
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, dense_retriever],
    weights=[0.4, 0.6]
)
ensemble_docs = ensemble_retriever.get_relevant_documents(query)
for i, doc in enumerate(ensemble_docs, 1):
    print(f"Ensemble Doc {i}: {doc.page_content} (Feature: {doc.metadata['feature_name']})")


=== Bước 3: Kết hợp tài liệu (Ensemble) ===
Ensemble Doc 1: Hiểu rõ hành vi và trải nghiệm khách hàng tại từng điểm chạm bằng phân tích chủ đề và xu hướng phản hồi. (Feature: Experience Insights)
Ensemble Doc 2: Tạo và triển khai chiến dịch chăm sóc khách hàng đa kênh để tăng tương tác và giữ chân khách hàng. (Feature: Customer Engagement Campaigns)
Ensemble Doc 3: Hệ thống quản lý yêu cầu hỗ trợ khách hàng với phân loại, ưu tiên và xử lý nhanh chóng nhờ AI. (Feature: Ticket Management System)
Ensemble Doc 4: Tập trung toàn bộ dữ liệu khách hàng và phân nhóm theo hành vi, giá trị, lịch sử tương tác. (Feature: Customer Database & Segmentation)
Ensemble Doc 5: Thu thập đánh giá khách hàng từ nhiều nền tảng như Google, Facebook và phân tích xu hướng, từ khóa và mức độ hài lòng. (Feature: Review Aggregation & Analysis)
Ensemble Doc 6: Thiết kế và gửi khảo sát qua Web, Mobile, Zalo, SMS, Email, QR, và POS để thu thập phản hồi từ khách hàng một cách linh hoạt. (Feature: Multichannel Survey 

In [9]:
#Rerank tài liệu bằng CrossEncoder
print("\n=== Bước 4: Rerank tài liệu ===")
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
pairs = [[query, doc.page_content] for doc in ensemble_docs]
scores = reranker.predict(pairs)

normalized_scores = [1 / (1 + np.exp(-score)) for score in scores]
scored_documents = sorted(zip(normalized_scores, ensemble_docs), key=lambda x: x[0], reverse=True)
reranked_docs = [doc for score, doc in scored_documents]

for i, (score, doc) in enumerate(scored_documents, 1):
    print(f"Reranked Doc {i} (score: {score:.4f}): {doc.page_content} (Feature: {doc.metadata['feature_name']})")



=== Bước 4: Rerank tài liệu ===


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Reranked Doc 1 (score: 0.8585): Hiểu rõ hành vi và trải nghiệm khách hàng tại từng điểm chạm bằng phân tích chủ đề và xu hướng phản hồi. (Feature: Experience Insights)
Reranked Doc 2 (score: 0.6744): Thu thập đánh giá khách hàng từ nhiều nền tảng như Google, Facebook và phân tích xu hướng, từ khóa và mức độ hài lòng. (Feature: Review Aggregation & Analysis)
Reranked Doc 3 (score: 0.4791): Tạo và triển khai chiến dịch chăm sóc khách hàng đa kênh để tăng tương tác và giữ chân khách hàng. (Feature: Customer Engagement Campaigns)
Reranked Doc 4 (score: 0.2935): Hệ thống quản lý yêu cầu hỗ trợ khách hàng với phân loại, ưu tiên và xử lý nhanh chóng nhờ AI. (Feature: Ticket Management System)
Reranked Doc 5 (score: 0.1717): Tập trung toàn bộ dữ liệu khách hàng và phân nhóm theo hành vi, giá trị, lịch sử tương tác. (Feature: Customer Database & Segmentation)
Reranked Doc 6 (score: 0.0964): Kết hợp AI và con người trong trung tâm liên lạc, giúp xử lý nhanh các yêu cầu, phản hồi tự động và gợi ý

In [10]:
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
    model="google/gemma-3n-e4b-it:free",
    default_headers={
        "HTTP-Referer": "<YOUR_SITE_URL>",
        "X-Title": "<YOUR_SITE_NAME>",
    },
    temperature= 0.7
)


context = "\n".join([f"Feature: {doc.metadata['feature_name']}\nDescription: {doc.page_content}" for doc in reranked_docs[:5]])

prompt = f"""
Bạn là một trợ lý chuyên gia tư vấn dịch vụ của Filum.ai.
Nhiệm vụ của bạn là phân tích "Pain Point" của người dùng và đề xuất các giải pháp phù hợp nhất từ Filum.ai.
Hãy đảm bảo câu trả lời được trình bày bằng ngôn ngữ mà người dùng đã sử dụng trong câu hỏi của họ.

Dưới đây là thông tin về các tính năng của Filum.ai, được sắp xếp từ **phù hợp nhất đến ít phù hợp hơn** với "Pain Point" của khách hàng:
{context}

"Pain Point" của khách hàng: {query}

**QUAN TRỌNG: Trước khi đưa ra câu trả lời cuối cùng, hãy thực hiện các bước suy luận sau đây. Không hiển thị các bước suy luận này trong phản hồi cuối cùng của bạn, chỉ hiển thị "Desired Output Format" sau khi suy luận xong.**

1.  **Phân tích Pain Point:** Đọc kỹ "Pain Point" và xác định các vấn đề cốt lõi mà khách hàng đang gặp phải. Các từ khóa quan trọng là gì? Vấn đề chính cần giải quyết là gì?
2.  **Đánh giá sự phù hợp của từng tính năng trong Context:**
    * Với mỗi tính năng trong "{context}", hãy tự hỏi: "Tính năng này giải quyết khía cạnh nào của 'Pain Point'? Nó có thực sự phù hợp không? Mức độ phù hợp là bao nhiêu (ví dụ: Rất cao, Cao, Trung bình, Thấp)?"
    * Lưu ý đến thứ tự đã được sắp xếp của các tính năng.
3.  **Lựa chọn giải pháp chính:** Dựa trên phân tích ở bước 2, hãy chọn ra **CHỈ MỘT (1) giải pháp tiêu biểu và hiệu quả nhất** mà bạn tin rằng giải quyết trực tiếp và mạnh mẽ nhất "Pain Point" của khách hàng. Giải thích ngắn gọn lý do tại sao đây là lựa chọn số 1.
4.  **Kiểm tra giải pháp bổ trợ (nếu có):** Xem xét xem có tính năng thứ hai nào trong "{context}" mà:
    * Cũng cực kỳ phù hợp với "Pain Point" (ví dụ: mức độ phù hợp Rất cao hoặc Cao).
    * VÀ bổ trợ, hỗ trợ giải pháp chính để giải quyết vấn đề một cách toàn diện hơn, chứ không phải chỉ là một giải pháp độc lập khác.
    * Nếu tìm thấy, giải thích ngắn gọn tại sao cả hai giải pháp này đều cần thiết và hữu ích khi kết hợp.
5.  **Tổng hợp và Trình bày:** Dựa trên các bước suy luận trên, chuẩn bị câu trả lời cuối cùng theo định dạng yêu cầu.

**Định dạng đầu ra mong muốn (sau các bước suy luận nội bộ của bạn):**
Potential Filum.ai Solutions:
- <Tên tính năng> – <Mô tả ngắn gọn cách giải pháp này giúp khắc phục vấn đề của khách hàng (Pain Point)>
- <Tên tính năng (nếu có giải pháp thứ hai)> – <Mô tả ngắn gọn cách giải pháp này giúp khắc phục vấn đề của khách hàng (Pain Point)>
"""



prompt = ChatPromptTemplate.from_template(prompt)
output_parser = StrOutputParser()

rag_chain = (
    {"context": RunnablePassthrough.assign(input=lambda x: context), "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [13]:
# Chạy chuỗi
print("\n=== Chạy chuỗi RAG và nhận phản hồi ===")
print("Pain Point:", query)
final_response = rag_chain.invoke({"query": query})

print(final_response)


=== Chạy chuỗi RAG và nhận phản hồi ===
Pain Point: Chúng tôi không có ý tưởng rõ ràng về điểm tiếp xúc của khách hàng đang gây ra sự thất vọng nhấ
Desired Output Format:
Potential Filum.ai Solutions:
- Experience Insights – Tính năng này sẽ giúp bạn hiểu rõ hành vi và trải nghiệm khách hàng tại từng điểm tiếp xúc, từ đó xác định chính xác điểm nào đang gây ra sự thất vọng nhất. Phân tích chủ đề và xu hướng phản hồi sẽ cung cấp cái nhìn sâu sắc về những vấn đề cụ thể mà khách hàng đang gặp phải.
- Review Aggregation & Analysis – Kết hợp với Experience Insights, tính năng này sẽ cung cấp thêm thông tin về những đánh giá khách hàng đang đưa ra trên các nền tảng khác nhau. Điều này giúp bạn xác định liệu sự thất vọng có liên quan đến những nhận xét tiêu cực trên mạng xã hội hay không, và từ đó đưa ra các biện pháp khắc phục phù hợp.
